In [7]:
import json
import numpy as np
import pandas as pd
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy, crf_viterbi_accuracy
from keras.models import load_model
from collections import defaultdict
from pprint import pprint
pd.set_option('max_columns', 600)
pd.set_option('max_rows', 500)

from data_utils import read_data,data_convert,tag_id,MAX_SEQ_LEN,id_tag
from albert_zh.extract_feature import BertVector

In [3]:
# 利用ALBERT提取文本特征
bert_model = BertVector(pooling_strategy="NONE", max_seq_len=MAX_SEQ_LEN)
word_encoder = lambda text: bert_model.encode([text])["encodes"][0]

# 载入模型
custom_objects = {'CRF': CRF, 'crf_loss': crf_loss, 'crf_viterbi_accuracy': crf_viterbi_accuracy}
model = load_model("albert_bilstm_crf.h5", custom_objects=custom_objects)

I:BERT_VEC:[graph:opt: 48]:model config: D:\学习\albert_BiLSTM_CRF\albert_zh/albert_tiny\albert_config_tiny.json
I:BERT_VEC:[graph:opt: 54]:build graph...


embedding_lookup_factorized. factorized embedding parameterization is used.
ln_type: postln
old structure of transformer.use: transformer_model,which use post-LN


I:BERT_VEC:[graph:opt:128]:load parameters from checkpoint...
I:BERT_VEC:[graph:opt:130]:freeze...
I:BERT_VEC:[graph:opt:133]:optimize...
I:BERT_VEC:[graph:opt:144]:write graph to a tmp file: ./tmp_graph11


In [ ]:
# 输入句子，一个句子一个句子进行预测
while 1:
    # 输入句子
    text = input("Please enter an sentence: ").replace(' ', '')
    # 利用训练好的模型进行预测
    train_x = np.array([word_encoder(text)])
    y = np.argmax(model.predict(train_x), axis=2)
    y = [id_tag[id] for id in y[0] if id]
    
    
    words=[w for w in text][:MAX_SEQ_LEN]
    tags=y[:len(words)]
    df=pd.DataFrame({'Sentence':words,'NER':tags})
    # 输出预测结果
    print(df)



Please enter an sentence: 驴妈妈旅游网创始人洪清华近日接受媒体采访谈及驴妈妈的发展模式时表示：现在，电商有两种做法——小而美的电商追求盈利，大而全的电商钟情规模。
   Sentence    NER
0         驴  B-ORG
1         妈  I-ORG
2         妈  I-ORG
3         旅  I-ORG
4         游  I-ORG
5         网  I-ORG
6         创      O
7         始      O
8         人      O
9         洪  B-PER
10        清  I-PER
11        华  I-PER
12        近      O
13        日      O
14        接      O
15        受      O
16        媒      O
17        体      O
18        采      O
19        访      O
20        谈      O
21        及      O
22        驴  B-LOC
23        妈      O
24        妈      O
25        的      O
26        发      O
27        展      O
28        模      O
29        式      O
30        时      O
31        表      O
32        示      O
33        ：      O
34        现      O
35        在      O
36        ，      O
37        电      O
38        商      O
39        有      O
40        两      O
41        种      O
42        做      O
43        法      O
44        —      O
45        —      O
46        小 